In [1]:
#imports
!pip install flask
from flask import Flask, request, render_template
import numpy as np
import threading
import webbrowser
import pickle
#import functions from our notebook
from utils import get_state_for_zip, input_county_zip
from datetime import datetime
import re
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import socket
import os
from sklearn.preprocessing import LabelEncoder

In [2]:
#import all the files from notebook
#import the model

#random forest model
with open('rf_model.pkl', 'rb') as f:
    model = pickle.load(f)





#import the scaler in this case minmax
scaler = MinMaxScaler()

# Cause injury
with open('cause_injury_category_encoder.pkl', 'rb') as f:
    cause_injury_category_encoder = pickle.load(f)
#Nature injury
with open('nature_injury_cateogory_encoder.pkl', 'rb') as f:
    nature_injury_category_encoder = pickle.load(f)
#Body 
with open('body_category_encoder.pkl', 'rb') as f:
    body_category_encoder = pickle.load(f)

#Carrier Type
with open('Carrier Type_encoder.pkl', 'rb') as f:
    carrier_type_encoder = pickle.load(f)

#Carrier Name
with open('Carrier Name_encoder.pkl', 'rb') as f:
    carrier_name_encoder = pickle.load(f)
    
#County Injury
with open('County of Injury_encoder.pkl', 'rb') as f:
    county_injury_encoder = pickle.load(f)

#District Name
with open('District Name_encoder.pkl', 'rb') as f:
    district_name_encoder = pickle.load(f)
    
#Medical Region
with open('Medical Fee Region_encoder.pkl', 'rb') as f:
    medical_fee_region_encoder = pickle.load(f)                       
    
#Gender
with open('gender_encoder.pkl', 'rb') as f:
       gender_encoder = pickle.load(f)
#State Zip 
with open('State_Zip_encoder.pkl', 'rb') as f:
       state_zip_encoder = pickle.load(f)
#County Zip 
with open('County_Zip_encoder.pkl', 'rb') as f:
       county_zip_encoder = pickle.load(f)


In [3]:
app = Flask(__name__)

myPort = 5007

#to encode the features
def encoder(mapping, key):
    key = str(key)
    if key in mapping:
        return mapping[key]  
    else:
        return 0

#to encode the codes
def codes(code, bins):
    for group, values in bins.items():
        if code in values:
            return group
    return None

#to extract the numbers from the codes
def extract_numbers(code):
    match = re.match(r'^(\d+)', code)
    return float(match.group(1)) if match else None

#to create the new feature related with the average_weekly_wage
def check_wage(average_weekly_wage):
    """Determine wage replacement eligibility based on average weekly wage."""
    return 0 if average_weekly_wage > 0 else 1
#to create the new feature
def check_retirement_status(age):
    """Determine retirement status based on age."""
    retirement_age = 62
    return 0 if age > retirement_age else 1
#to create the new feature
def check_notification_status(declaration_time):
    """Determine notification status based on declaration time in days."""
    return 1 if declaration_time <= 30 else 0
#to return the first hearing date into a binary variable
def check_hearing_date(first_hearing_date):
    if first_hearing_date:
        return 1  # Se o valor for preenchido (não nulo, não vazio)
    else:
        return 0

# Add the label_mapping dictionary
label_mapping = {
    0: '1. CANCELLED',
    1: '2. NON-COMP',
    2: '3. MED ONLY',
    3: '4. TEMPORARY',
    4: '5. PPD SCH LOSS',
    5: '6. PPD NSL',
    6: '7. PTD',
    7: '8. DEATH'
}


# Function to make the flask 
def run_app():
    app.run('localhost', debug=True, port=myPort, use_reloader=False)


# route to home page
@app.route('/')
def home():
    return render_template('home.html')

# Route to process data from the website forms 
@app.route('/predict', methods=['GET', 'POST'])
def predict():
    if request.method == 'POST':
        try:
            # Coleta os dados do formulário
            accident_date = request.form['accident-date']
            age_at_injury = float(request.form['age-at-injury'])
            alternative_dispute_resolution = request.form['alternative-dispute-resolution']
            assembly_date = request.form['assembly-date']
            attorney_representative = request.form['attorney-representative']
            average_weekly_wage = float(request.form['average-weekly-wage'])
            birth_year = int(request.form['birth-year'])
            c2_date = request.form['c2-date']
            c3_date = request.form['c3-date']
            carrier_type = request.form['carrier-type'].upper()
            carrier_name = request.form['carrier-name'].upper()
            county_of_injury = request.form['county-of-injury'].upper()
            covid19_indicator = request.form['agreement-reached']
            district_name = request.form['district-name'].upper()
            first_hearing_date = request.form['first-hearing-date']
            gender = request.form['gender']
            ime4_count = int(request.form['ime4-count'])
            industry_code = request.form['industry-code']
            medical_region = request.form['medical-region']
            wcio_cause_of_injury_code = request.form['wcio-cause-of-injury-code']
            wcio_nature_of_injury_code = request.form['wcio-nature-of-injury-code']
            wcio_part_of_body_code = request.form['wcio-part-of-body-code']
            zip_code = request.form['zip']
            agreement_reached = request.form['agreement-reached']
            wcb_decision = request.form['wcb-decision']
            number_dependents = int(request.form['number_dependents'])

            # return the features dates to datetime type
            accident_date = accident_date.strip()
            accident_date = pd.to_datetime(accident_date, format='%Y-%m-%d')
            assembly_date = assembly_date.strip()
            assembly_date = pd.to_datetime(assembly_date, format='%Y-%m-%d')
            c2_date = c2_date.strip()
            c2_date = pd.to_datetime(c2_date, format='%Y-%m-%d')
            c3_date = c3_date.strip()
            c3_date = pd.to_datetime(c3_date, format='%Y-%m-%d')
            first_hearing_date = pd.to_datetime(first_hearing_date, format = '%Y-%m-%d')
            #returning the first hearing date to the new type
            first_hearing_date = check_hearing_date(first_hearing_date)
                   
            #creating the new features
            log_ime_4_count = np.log1p(ime4_count)
            log_average_weekly_wage = np.log1p(average_weekly_wage)
            
            # New Features (using datetime objects)
            declaration_time = (assembly_date - accident_date).days
            receiving_time = (c2_date - assembly_date).days
            receiving_time_c2 = (c2_date - accident_date).days
            wage_replacement_eligibility = check_wage(average_weekly_wage)
            retirement_status = check_retirement_status(age_at_injury)
            notification_status = check_notification_status(declaration_time)
            zip_state = get_state_for_zip(zip_code)
            zip_county = input_county_zip(zip_code)
    
            ###ENCODING BINARY VARIABLES
            # Mapping 'Y'/'yes' to 1, 'N'/'no' to 0
            alternative_dispute_resolution = 1 if alternative_dispute_resolution.lower() in ['y', 'yes'] else 0
            attorney_representative = 1 if attorney_representative.lower() in ['y', 'yes'] else 0
            covid19_indicator = 1 if covid19_indicator.lower() in ['y', 'yes'] else 0
            agreement_reached = 1 if agreement_reached.lower() in ['y', 'yes'] else 0
    
    
            ###CODES
            industry_code_number = extract_numbers(industry_code)
            cause_of_injury_number = extract_numbers(wcio_cause_of_injury_code)
            nature_of_injury_number = extract_numbers(wcio_nature_of_injury_code)
            part_of_body_number = extract_numbers(wcio_part_of_body_code)
    
    
            #features_to_encode = [cause_of_injury_number, nature_of_injury_number, carrier_name, part_of_body_number,  carrier_type, county_of_injury,
             #                     district_name,district_name, gender, medical_region]

            #encoding all the features through their file encoder 
            cause_injury_category = codes(cause_of_injury_number, cause_injury_category_encoder)
            nature_injury_category = codes(nature_of_injury_number, nature_injury_category_encoder)
            body_category = codes(part_of_body_number, body_category_encoder)
            carrier_name = encoder(carrier_name_encoder , carrier_name)
            carrier_type = encoder(carrier_type_encoder,carrier_type )
            county_of_injury = encoder(county_injury_encoder, county_of_injury)
            district_name= encoder(district_name_encoder, district_name)
            gender = encoder(gender_encoder, gender)
            medical_region = encoder(medical_fee_region_encoder, medical_region)
            zip_state = encoder(state_zip_encoder, zip_state) 
            zip_county = encoder(county_zip_encoder, zip_county)
            
    
            
            numerical_features = np.array([[age_at_injury, alternative_dispute_resolution, attorney_representative, 
                                            average_weekly_wage, covid19_indicator,carrier_type,  gender, medical_region, county_of_injury, district_name,
                                            ime4_count, industry_code_number,  cause_of_injury_number,
                                            nature_of_injury_number, part_of_body_number, number_dependents, declaration_time, first_hearing_date,
                                            receiving_time,  wage_replacement_eligibility, retirement_status, carrier_name,
                                            notification_status, log_ime_4_count, log_average_weekly_wage,
                                            cause_injury_category, nature_injury_category, body_category, zip_state, zip_county]])
    
            
            # Scale the numerical features using the loaded scaler
            numerical_features = numerical_features.astype(float)
    
            scaler.fit(numerical_features)
    
            scaled_numerical_features = scaler.transform(numerical_features)
            
            # Combine the scaled features with the categorical feature
            final_features = np.array([[age_at_injury, carrier_name,
                                       attorney_representative, average_weekly_wage, alternative_dispute_resolution, carrier_type, 
                                        county_of_injury,district_name, medical_region,
                                     covid19_indicator,  first_hearing_date, ime4_count, 
                                       industry_code_number, cause_of_injury_number, nature_of_injury_number, 
                                       part_of_body_number, 
                                       declaration_time, receiving_time, #receiving_time_c2, 
                                        wage_replacement_eligibility, #retirement_status, notification_status, zip_code,
                                        cause_injury_category, body_category]])
            # Make the prediction
            prediction = model.predict(final_features)[0]
    
            # Decode the prediction using the label_mapping dictionary
            decision = label_mapping.get(prediction, "Unknown Prediction")
            

            return render_template('result.html', decision=decision)
        
        except Exception as e:
            return f"Error: {str(e)}"
    

    return render_template('home.html')

threading.Thread(target=run_app).start()

# Openning the app in webbrowser
webbrowser.open(f"http://127.0.0.1:{myPort}")


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://localhost:5007
Press CTRL+C to quit


True